In [0]:
# !pip3 install http://download.pytorch.org/whl/cu80/torch-0.4.1-cp36-cp36m-linux_x86_64.whl 
# !pip3 install torchvision

In [25]:
import torch
import torch.functional as F
import torch.nn as nn
import torch.optim as optim
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import unicodedata
import re
import time

print(torch.__version__)

1.3.1


In [26]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [0]:
topics = []
jokes = []
with open('/gdrive/My Drive/train_data2.txt', encoding='ISO-8859-1') as f1:
  for line in f1:
    jokes.append(line[:-1])
with open('/gdrive/My Drive/pos_tags2.txt', encoding='ISO-8859-1') as f2:
  for line in f2:
    topics.append(line[:-1])

In [28]:

num_examples = 15783 

# creates lists containing each pair
original_word_pairs = [[topics[i], jokes[i]] for i in range(num_examples)]
data = pd.DataFrame(original_word_pairs, columns=["topics", "jokes"])
data.head(5)


,topics,jokes
0,hey guy twitter,"yes , i know what you guys are thinking , "" he..."
1,i m glad cable truth i ve talk show host cable i,"i ' m glad to be on cable . the truth is , i '..."
2,i m tbs demographicpeople afford hbo,i ' m happy to report that we ' re already # 1...
3,night show channel lot money viewers trouble f...,it ' s not easy doing a late - night show on a...
4,man show business blackest channel tv,that ' s rightthe whitest man in show busines...


In [0]:
def unicode_to_ascii(s):
    """
    Normalizes latin chars with accent to their canonical decomposition
    """
    return ''.join(c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn')

def preprocess_sentence(w):
    w = unicode_to_ascii(w.lower().strip())
    
    # creating a space between a word and the punctuation following it
    # eg: "he is a boy." => "he is a boy ." 
    # Reference:- https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
    w = re.sub(r"([?.!,¿])", r" \1 ", w)
    w = re.sub(r'[" "]+', " ", w)
    
    # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
    w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)
    
    w = w.rstrip().strip()
    
    # adding a start and an end token to the sentence
    # so that the model know when to start and stop predicting.
    w = '<start> ' + w + ' <end>'
    return w

##Data Exploration

In [30]:
# Now we do the preprocessing using pandas and lambdas
data["topics"] = data.topics.apply(lambda w: preprocess_sentence(w))
data["jokes"] = data.jokes.apply(lambda w: preprocess_sentence(w))
data.sample(10)

,topics,jokes
10472,<start> sister birth state art delivery room t...,<start> my sister gave birth in a state of the...
8713,<start> donald trump time magazine person year...,<start> donald trump has been named time magaz...
5284,<start> mother day moms hooters s way mother s...,"<start> on mother s day , moms can eat for fre..."
12252,<start> yo mama people exercize <end>,<start> yo mama is so fat people run around he...
9360,<start> invention people kernels corn face fir...,"<start> before the invention of popcorn , peop..."
10968,<start> lord strength i clutches cholesterol p...,"<start> lord , grant me the strength that i ma..."
5836,<start> weekend george clooney italy bachelor ...,"<start> over the weekend , george clooney got ..."
3711,<start> tremor yesterday la earthquake asia <end>,<start> felt a big tremor yesterday in la . th...
14151,<start> penis condom m <end>,<start> what did the penis say to the condom ?...
15183,<start> definition people country <end>,<start> definition of alien people from anothe...


In [0]:
# This class creates a word -> index mapping (e.g,. "dad" -> 5) and vice-versa 
# (e.g., 5 -> "dad") for each language,
class LanguageIndex():
    def __init__(self, lang):
        """ lang are the list of phrases from each language"""
        self.lang = lang
        self.word2idx = {}
        self.idx2word = {}
        self.vocab = set()
        
        self.create_index()
        
    def create_index(self):
        for phrase in self.lang:
            # update with individual tokens
            self.vocab.update(phrase.split(' '))
            
        # sort the vocab
        self.vocab = sorted(self.vocab)

        # add a padding token with index 0
        self.word2idx['<pad>'] = 0
        
        # word to index mapping
        for index, word in enumerate(self.vocab):
            self.word2idx[word] = index + 1 # +1 because of pad token
        
        # index to word mapping
        for word, index in self.word2idx.items():
            self.idx2word[index] = word

In [32]:
# index language using the class above
inp_lang = LanguageIndex(data["topics"].values.tolist())
targ_lang = LanguageIndex(data["jokes"].values.tolist())
# Vectorize the input and target languages
input_tensor = [[inp_lang.word2idx[s] for s in topic.split(' ')]  for topic in data["topics"].values.tolist()]
target_tensor = [[targ_lang.word2idx[s] for s in joke.split(' ')]  for joke in data["jokes"].values.tolist()]
input_tensor[15:25]

[[3, 12366, 2396, 388, 6449, 10543, 7061, 10508, 2],
 [3, 1968, 3108, 11362, 9794, 3307, 9743, 4832, 12897, 11362, 12473, 1077, 2],
 [3,
  8614,
  10879,
  11801,
  13012,
  740,
  13965,
  1287,
  739,
  1056,
  4173,
  13012,
  10879,
  9321,
  2],
 [3, 1586, 9895, 13895, 5220, 12083, 12057, 9269, 1101, 2],
 [3, 12366, 7990, 6464, 9727, 12366, 7747, 4287, 2],
 [3, 4639, 6895, 7681, 3193, 7681, 997, 12156, 6203, 5809, 6895, 2],
 [3, 11647, 472, 12063, 13363, 2683, 2299, 2299, 9419, 13729, 9419, 2],
 [3, 14112, 5081, 12147, 2295, 12881, 14004, 6917, 2],
 [3, 13691, 9764, 11399, 3928, 9389, 13765, 8244, 3495, 2990, 10879, 6603, 2],
 [3, 4221, 10564, 10879, 3766, 10564, 13625, 2879, 615, 2]]

In [33]:
target_tensor[:10]

[[6,
  20324,
  3,
  8828,
  10018,
  19901,
  20352,
  7939,
  845,
  18257,
  3,
  8365,
  3,
  9484,
  15572,
  18197,
  7937,
  7184,
  18917,
  4,
  5],
 [6,
  8828,
  10792,
  7524,
  18427,
  1491,
  12554,
  2548,
  4,
  18197,
  18811,
  9454,
  3,
  8828,
  19385,
  5450,
  12475,
  1596,
  8,
  17919,
  16319,
  8638,
  12554,
  1435,
  2548,
  6194,
  16440,
  8828,
  19742,
  4,
  5],
 [6,
  8828,
  10792,
  8101,
  18427,
  15023,
  18193,
  19799,
  14636,
  509,
  9007,
  18009,
  15572,
  9891,
  4768,
  13225,
  19967,
  2641,
  17870,
  314,
  8205,
  4,
  5],
 [6,
  9484,
  15572,
  12291,
  5678,
  5301,
  8,
  10182,
  12181,
  16319,
  12554,
  8,
  2977,
  20110,
  8,
  10691,
  12475,
  11675,
  616,
  18193,
  19503,
  8183,
  18778,
  6736,
  4,
  16764,
  18193,
  15572,
  19988,
  8828,
  10302,
  12030,
  4,
  5],
 [6,
  18193,
  15572,
  15281,
  18197,
  19962,
  10914,
  9007,
  16319,
  2493,
  9454,
  1226,
  12554,
  18197,
  15938,
  1829,
  2977,
 

In [34]:
def max_length(tensor):
    return max(len(t) for t in tensor)

# calculate the max_length of input and output tensor
max_length_inp, max_length_tar = max_length(input_tensor), max_length(target_tensor)
(max_length_inp, max_length_tar)

(45, 98)

In [35]:
def pad_sequences(x, max_len):
    padded = np.zeros((max_len), dtype=np.int64)
    if len(x) > max_len: padded[:] = x[:max_len]
    else: padded[:len(x)] = x
    return padded
input_tensor = [pad_sequences(x, max_length_inp) for x in input_tensor]
target_tensor = [pad_sequences(x, max_length_tar) for x in target_tensor]
len(input_tensor)

15783

In [36]:
# Creating training and validation sets using an 80-20 split
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.2)

# Show length
len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val)

(12626, 12626, 3157, 3157)

##Load data into DataLoader for Batching

In [0]:
from torch.utils.data import Dataset, DataLoader
class MyData(Dataset):
    def __init__(self, X, y):
        self.data = X
        self.target = y
        # TODO: convert this into torch code is possible
        self.length = [ np.sum(1 - np.equal(x, 0)) for x in X]
        
    def __getitem__(self, index):
        x = self.data[index]
        y = self.target[index]
        x_len = self.length[index]
        return x,y,x_len
    
    def __len__(self):
        return len(self.data)

##Parameters

In [0]:
BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 64
N_BATCH = BUFFER_SIZE//BATCH_SIZE
embedding_dim = 256
units = 1024
vocab_inp_size = len(inp_lang.word2idx)
vocab_tar_size = len(targ_lang.word2idx)

train_dataset = MyData(input_tensor_train, target_tensor_train)
val_dataset = MyData(input_tensor_val, target_tensor_val)

dataset = DataLoader(train_dataset, batch_size = BATCH_SIZE, 
                     drop_last=True,
                     shuffle=True)

In [0]:

class Encoder(nn.Module):
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
        super(Encoder, self).__init__()
        self.batch_sz = batch_sz
        self.enc_units = enc_units
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.embedding = nn.Embedding(self.vocab_size, self.embedding_dim)
        self.gru = nn.GRU(self.embedding_dim, self.enc_units)
        
    def forward(self, x, lens, device):
        # x: batch_size, max_length 
        
        # x: batch_size, max_length, embedding_dim
        x = self.embedding(x) 
                
        # x transformed = max_len X batch_size X embedding_dim
        # x = x.permute(1,0,2)
        x = pack_padded_sequence(x, lens) # unpad
    
        self.hidden = self.initialize_hidden_state(device)
        
        # output: max_length, batch_size, enc_units
        # self.hidden: 1, batch_size, enc_units
        output, self.hidden = self.gru(x, self.hidden) # gru returns hidden state of all timesteps as well as hidden state at last timestep
        
        # pad the sequence to the max length in the batch
        output, _ = pad_packed_sequence(output)
        
        return output, self.hidden

    def initialize_hidden_state(self, device):
        return torch.zeros((1, self.batch_sz, self.enc_units)).to(device)

In [0]:
### sort batch function to be able to use with pad_packed_sequence
def sort_batch(X, y, lengths):
    lengths, indx = lengths.sort(dim=0, descending=True)
    X = X[indx]
    y = y[indx]
    return X.transpose(0,1), y, lengths # transpose (batch x seq) to (seq x batch)

##Testing the Encoder

In [41]:
### Testing Encoder part
# TODO: put whether GPU is available or not
# Device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)

encoder.to(device)
# obtain one sample from the data iterator
it = iter(dataset)
x, y, x_len = next(it)

# sort the batch first to be able to use with pac_pack_sequence
xs, ys, lens = sort_batch(x, y, x_len)

enc_output, enc_hidden = encoder(xs.to(device), lens, device)

print(enc_output.size()) # max_length, batch_size, enc_units

torch.Size([26, 64, 1024])


In [0]:
class Decoder(nn.Module):
    def __init__(self, vocab_size, embedding_dim, dec_units, enc_units, batch_sz):
        super(Decoder, self).__init__()
        self.batch_sz = batch_sz
        self.dec_units = dec_units
        self.enc_units = enc_units
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.embedding = nn.Embedding(self.vocab_size, self.embedding_dim)
        self.gru = nn.GRU(self.embedding_dim + self.enc_units, 
                          self.dec_units,
                          batch_first=True)
        self.fc = nn.Linear(self.enc_units, self.vocab_size)
        
        # used for attention
        self.W1 = nn.Linear(self.enc_units, self.dec_units)
        self.W2 = nn.Linear(self.enc_units, self.dec_units)
        self.V = nn.Linear(self.enc_units, 1)
    
    def forward(self, x, hidden, enc_output):
        # enc_output original: (max_length, batch_size, enc_units)
        # enc_output converted == (batch_size, max_length, hidden_size)
        enc_output = enc_output.permute(1,0,2)
        # hidden shape == (batch_size, hidden size)
        # hidden_with_time_axis shape == (batch_size, 1, hidden size)
        # we are doing this to perform addition to calculate the score
        
        # hidden shape == (batch_size, hidden size)
        # hidden_with_time_axis shape == (batch_size, 1, hidden size)
        hidden_with_time_axis = hidden.permute(1, 0, 2)
        
        # score: (batch_size, max_length, hidden_size)
        score = torch.tanh(self.W1(enc_output) + self.W2(hidden_with_time_axis))
        
        #score = torch.tanh(self.W2(hidden_with_time_axis) + self.W1(enc_output))
          
        # attention_weights shape == (batch_size, max_length, 1)
        # we get 1 at the last axis because we are applying score to self.V
        attention_weights = torch.softmax(self.V(score), dim=1)
        
        # context_vector shape after sum == (batch_size, hidden_size)
        context_vector = attention_weights * enc_output
        context_vector = torch.sum(context_vector, dim=1)
        
        # x shape after passing through embedding == (batch_size, 1, embedding_dim)
        x = self.embedding(x)
        
        # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
        #x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)
        x = torch.cat((context_vector.unsqueeze(1), x), -1)
        
        # passing the concatenated vector to the GRU
        # output: (batch_size, 1, hidden_size)
        output, state = self.gru(x)
        
        
        # output shape == (batch_size * 1, hidden_size)
        output =  output.view(-1, output.size(2))
        
        # output shape == (batch_size * 1, vocab)
        x = self.fc(output)
        
        return x, state, attention_weights
    
    def initialize_hidden_state(self):
        return torch.zeros((1, self.batch_sz, self.dec_units))

In [43]:
# Device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)

encoder.to(device)
# obtain one sample from the data iterator
it = iter(dataset)
x, y, x_len = next(it)

print("Input: ", x.shape)
print("Output: ", y.shape)

# sort the batch first to be able to use with pac_pack_sequence
xs, ys, lens = sort_batch(x, y, x_len)

enc_output, enc_hidden = encoder(xs.to(device), lens, device)
print("Encoder Output: ", enc_output.shape) # batch_size X max_length X enc_units
print("Encoder Hidden: ", enc_hidden.shape) # batch_size X enc_units (corresponds to the last state)

decoder = Decoder(vocab_tar_size, embedding_dim, units, units, BATCH_SIZE)
decoder = decoder.to(device)

#print(enc_hidden.squeeze(0).shape)

dec_hidden = enc_hidden#.squeeze(0)
dec_input = torch.tensor([[targ_lang.word2idx['<start>']]] * BATCH_SIZE)
print("Decoder Input: ", dec_input.shape)
print("--------")

for t in range(1, y.size(1)):
    # enc_hidden: 1, batch_size, enc_units
    # output: max_length, batch_size, enc_units
    predictions, dec_hidden, _ = decoder(dec_input.to(device), 
                                         dec_hidden.to(device), 
                                         enc_output.to(device))
    
    print("Prediction: ", predictions.shape)
    print("Decoder Hidden: ", dec_hidden.shape)
    
    #loss += loss_function(y[:, t].to(device), predictions.to(device))
    
    dec_input = y[:, t].unsqueeze(1)
    print(dec_input.shape)
    break

Input:  torch.Size([64, 45])
Output:  torch.Size([64, 98])
Encoder Output:  torch.Size([33, 64, 1024])
Encoder Hidden:  torch.Size([1, 64, 1024])
Decoder Input:  torch.Size([64, 1])
--------
Prediction:  torch.Size([64, 20437])
Decoder Hidden:  torch.Size([1, 64, 1024])
torch.Size([64, 1])


In [0]:
criterion = nn.CrossEntropyLoss()

def loss_function(real, pred):
    """ Only consider non-zero inputs in the loss; mask needed """
    #mask = 1 - np.equal(real, 0) # assign 0 to all above 0 and 1 to all 0s
    #print(mask)
    mask = real.ge(1).type(torch.cuda.FloatTensor)
    
    loss_ = criterion(pred, real) * mask 
    return torch.mean(loss_)

In [45]:
# Device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## TODO: Combine the encoder and decoder into one class
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)
decoder = Decoder(vocab_tar_size, embedding_dim, units, units, BATCH_SIZE)

encoder.to(device)
decoder.to(device)

optimizer = optim.Adam(list(encoder.parameters()) + list(decoder.parameters()), 
                       lr=0.001)
device

device(type='cuda', index=0)

##Train


In [46]:
EPOCHS = 90

for epoch in range(EPOCHS):
    start = time.time()
    
    encoder.train()
    decoder.train()
    
    total_loss = 0
    
    for (batch, (inp, targ, inp_len)) in enumerate(dataset):
        loss = 0
        
        xs, ys, lens = sort_batch(inp, targ, inp_len)
        enc_output, enc_hidden = encoder(xs.to(device), lens, device)
        dec_hidden = enc_hidden
        dec_input = torch.tensor([[targ_lang.word2idx['<start>']]] * BATCH_SIZE)
        
        for t in range(1, ys.size(1)):
            predictions, dec_hidden, _ = decoder(dec_input.to(device), 
                                         dec_hidden.to(device), 
                                         enc_output.to(device))
            loss += loss_function(ys[:, t].to(device), predictions.to(device))
            #loss += loss_
            dec_input = ys[:, t].unsqueeze(1)
            
        
        batch_loss = (loss / int(ys.size(1)))
        total_loss += batch_loss
        
        optimizer.zero_grad()
        
        loss.backward()

        ### UPDATE MODEL PARAMETERS
        optimizer.step()
        
        if batch % 100 == 0:
            print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                         batch,
                                                         batch_loss.detach().item()))
        
    if epoch%5==0:
      path1 = F"/gdrive/My Drive/{'encoder2.torch'}" 
      torch.save(encoder, path1)
      path2 = F"/gdrive/My Drive/{'decoder2.torch'}" 
      torch.save(decoder, path2)
    ### TODO: Save checkpoint for model
    print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                        total_loss / N_BATCH))
    print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

Epoch 1 Batch 0 Loss 3.6868
Epoch 1 Batch 100 Loss 1.3537


/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Encoder. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Embedding. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type GRU. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Decoder. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-

Epoch 1 Loss 1.6303
Time taken for 1 epoch 114.65635681152344 sec

Epoch 2 Batch 0 Loss 1.3097
Epoch 2 Batch 100 Loss 1.4145
Epoch 2 Loss 1.3012
Time taken for 1 epoch 114.4038622379303 sec

Epoch 3 Batch 0 Loss 1.0324
Epoch 3 Batch 100 Loss 1.1155
Epoch 3 Loss 1.0942
Time taken for 1 epoch 114.39831829071045 sec

Epoch 4 Batch 0 Loss 0.9261
Epoch 4 Batch 100 Loss 0.9360
Epoch 4 Loss 0.9160
Time taken for 1 epoch 114.6073169708252 sec

Epoch 5 Batch 0 Loss 0.7102
Epoch 5 Batch 100 Loss 0.6907
Epoch 5 Loss 0.7755
Time taken for 1 epoch 114.13746166229248 sec

Epoch 6 Batch 0 Loss 0.6721
Epoch 6 Batch 100 Loss 0.6963
Epoch 6 Loss 0.6650
Time taken for 1 epoch 114.54607081413269 sec

Epoch 7 Batch 0 Loss 0.5704
Epoch 7 Batch 100 Loss 0.5761
Epoch 7 Loss 0.5677
Time taken for 1 epoch 114.32960677146912 sec

Epoch 8 Batch 0 Loss 0.4485
Epoch 8 Batch 100 Loss 0.5413
Epoch 8 Loss 0.4866
Time taken for 1 epoch 114.19988322257996 sec

Epoch 9 Batch 0 Loss 0.3973
Epoch 9 Batch 100 Loss 0.4391
Ep

In [47]:
path1 = F"/gdrive/My Drive/{'encoder2.torch'}" 
torch.save(encoder, path1)
path2 = F"/gdrive/My Drive/{'decoder2.torch'}" 
torch.save(decoder, path2)

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Encoder. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Embedding. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type GRU. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Decoder. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-

##Generation


In [0]:
# path1 = F"/gdrive/My Drive/{'encoder.torch'}" 
# torch.save(encoder, path1)
# path2 = F"/gdrive/My Drive/{'decoder.torch'}" 
# torch.save(decoder, path2)

In [94]:
import random
for i in range(20):
      select=random.sample(range(10307),3)
      xs=[3]+select+[2]
      print([inp_lang.idx2word[i] for i in xs])

      xs = [ xs for i in range(BATCH_SIZE)]
      xs=torch.LongTensor(xs)
      xs=xs.t()
      lens=len(xs)
      lens = [ lens for i in range(BATCH_SIZE)]
      lens=torch.LongTensor(lens)
      #print(xs.size(),lens.size())

      enc_output, enc_hidden = encoder(xs.to(device), lens, device)
      dec_hidden = enc_hidden
      dec_input = torch.tensor([[targ_lang.word2idx['<start>']]] * BATCH_SIZE)
      res=[]
      for t in range(1, ys.size(1)):
        predictions, dec_hidden, _ = decoder(dec_input.to(device), 
                                          dec_hidden.to(device), 
                                          enc_output.to(device))

        dec_input = predictions.argmax(dim=1).unsqueeze(1)
        p=predictions.argmax(dim=1).tolist()
        res.append(targ_lang.idx2word[p[0]])

      sentence=''
      for i in res:
        if i!='<end>' and i!='<pad>':
          sentence+= i+' '
      print(sentence)

['<start>', 'hulk', 'copenhagen', 'promotion', '<end>']
what did the like a fancy french of a what promotion . 
['<start>', 'chancellor', 'cockatoo', 'bocelli', '<end>']
so , a german has unveiled a free has officially confirmed the first has officially confirmed the medicine has officially confirmed the medicine has officially confirmed the medicine has officially confirmed the medicine has officially confirmed the medicine has officially confirmed the medicine has officially confirmed the medicine has officially confirmed the medicine has officially confirmed the medicine has officially confirmed the medicine has officially confirmed the medicine has officially confirmed the medicine has officially confirmed the medicine has officially confirmed the medicine has officially confirmed the medicine has officially confirmed the medicine has officially confirmed the 
['<start>', 'doorway', 'baskets', 'mannequin', '<end>']
why do we all know who knew that i like to call it was trying to ca

In [246]:
#xs=[inp_lang.word2idx[i] for i in topic_words.split()]
xs = [ xs for i in range(BATCH_SIZE)]
xs=torch.LongTensor(xs)
xs=xs.t()
lens=len(xs)
lens = [ lens for i in range(BATCH_SIZE)]
lens=torch.LongTensor(lens)

xs.size()


torch.Size([7, 64])

In [0]:
enc_output, enc_hidden = encoder(xs.to(device), lens, device)
dec_hidden = enc_hidden
dec_input = torch.tensor([[targ_lang.word2idx['<start>']]] * BATCH_SIZE)
res=[]
for t in range(1, ys.size(1)):
  predictions, dec_hidden, _ = decoder(dec_input.to(device), 
                                    dec_hidden.to(device), 
                                    enc_output.to(device))

  dec_input = predictions.argmax(dim=1).unsqueeze(1)
  p=predictions.argmax(dim=1).tolist()
  res.append(targ_lang.idx2word[p[0]])

In [248]:
sentence=''
for i in res:
  if i!='<end>' and i!='<pad>':
    sentence+= i+' '
sentence

'i can tell me some of . '

In [101]:
jo=[]
for i in ys[0].tolist():
  jo.append(targ_lang.idx2word[i])
jo

['<start>',
 'marcus',
 'bachmann',
 'wrote',
 'an',
 'open',
 'letter',
 'to',
 'conservatives',
 'describing',
 'his',
 'wife',
 'michele',
 'as',
 'rock',
 'solid',
 '.',
 'probably',
 'not',
 'helping',
 'was',
 'that',
 'he',
 'then',
 'wrote',
 ',',
 'as',
 'rock',
 'solid',
 'as',
 'taylor',
 'lautner',
 's',
 'yummy',
 'abs',
 '.',
 '<end>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>']